# Identifikacija zvezde

Proces identifikacije zvezde poteka z uporabo <a href="https://unilj-my.sharepoint.com/:b:/g/personal/aleksander_grm_fpp_uni-lj_si/EQhyqtLl3lNKsl2Jo21S4f8Bidd2DuJ_hLI1jO1bR5Cg8w">Navtičnega almanaha</a> (*Dec* in *Sha* zvezde) in meritvijo višine $H$ ter azimuta $\omega$ zvezde.

Poleg same meritve moramo določiti še čas meritve in položaj meritve. Položaje vnašamo v obliki, kot je v navtiki dogovor 

$\varphi = \#\#^\circ \, \#\#.\#' \, \mathrm{N/S}$, $\lambda = \#\#\#^\circ \, \#\#.\#' \, \mathrm{E/W}$, 

kjer imamo stopinje kot celo število in minute kot število na 1 decimalno mesto natančno.

Format vnosa položaja:
- LAT-$\varphi$ in LONG-$\lambda$ vnašamo v formatu [stopinje, minute, stran neba]

Primer vnosa položaja:
- $\varphi$: [12, 34.5, 'N']
- $\lambda$: [117, 12.6, 'W']
- stran neba mora biti v **enojnih navednicah**, kjer so dovoljeni znaki 'N', 'S', 'E', 'W'

Čas meritve zapišemo v obliki
- datum: *date*=[day,month,year]
- ura: *time*=[hour,minutes, seconds]

Višino in azumut zapišemo
- višina: $h=\#\#^\circ \#\#'$=[deg,min]
- azimut: $\omega=\#\#\#^\circ$=[deg]

<hr>

**Postopek identifikacije zvezde**

Določimo vhode parametre:
1. Določimo naš položaj meritve $P_0$
2. Določimo čas meritve *date*, *time*
3. Izmerimo višino nebesnega telesa s sekstantom $H$
4. Izmerimo azimut do nebesnega telesa $\omega$

Nato pričnemo z izračunom, ki poteka v sledečih korakih:
1. pretvorimo izmerjene količine:
\begin{align*}
    Z & = 90^\circ - H \\
    \psi & = 90^\circ - \varphi
\end{align*}

2. izračunamo razdaljo $p$ in nato deklinacijo $\delta$ izmeerjene zvezde
\begin{align*}
    \cos p & = \cos Z \cos \psi + \sin Z \sin \psi \cos \omega\\
    \delta & = 90^\circ - p
\end{align*}

3. določimo mestni časovni kot zvezde $\mathrm{Lha}$
$$
\cos \mathrm{Lha} = \frac{\cos Z - \cos p \cos \psi}{\sin p \sin \psi}
$$
kjer imamo pogoj, če je $\omega < 180^\circ$ sledi, da je potrebno izračunani $\mathrm{Lha}$ popraviti in sicer sledi $\to$ $\mathrm{Lha} = 360^\circ - \mathrm{Lha}$

4. s pomočjo $\mathrm{Lha}$ lahko sedaj določimo še siderični časovni kot zvezde $\mathrm{Sha}$
\begin{align*}
    \mathrm{Lha}_{\gamma} & = \mathrm{Gha}_{\gamma} + (\pm \lambda)\\
    \mathrm{Sha} & = \mathrm{Lha} - \mathrm{Lha}_{\gamma}
\end{align*}
kjer imamo pogoj, če je $\mathrm{Sha} < 0^\circ$ sledi, da je potrebno izračunani $\mathrm{Sha}$ popraviti in sicer sledi $\to$ $\mathrm{Sha} = 360^\circ + \mathrm{Sha}$

Sedaj ko imamo določeno deklinacijo zvezde $\delta$ in njen siderični časovni kot $\mathrm{Sha}$ lahko s pomočjo *Navtičnega almanaha* identificiramo ime zvezde.

In [1]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

# needed by basemap
os.environ['PROJ_LIB'] = '/opt/miniconda3/envs/astrolab/share/proj'

In [2]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [3]:
# poišče vse vidne zvezde nad obzorjem
def findAllVisibleStars(t,pos,ns_db):
    
    star_ids = ns_db.keys()
    h0 = 0
    h1 = 90
    
    vs = []
    for s_id in star_ids:
        sdd = cd.get_star_data(s_id,t)
        saz = cd.get_star_altaz(s_id,t,pos)
        if saz['alt'] > h0 and saz['alt'] < h1:
            s_name = ns_db[s_id][0]
            vs.append({'name':s_name, 'dec':sdd['dec'], 'sha':sdd['sha'], 'alt':saz['alt'], 'az':saz['az']})
    
    return vs

In [4]:
# Find star dec and sha
def getStarDecAndSha(s_h,s_w,pos,date,time):
    
    t = [date[2],date[1],date[0],time[0],time[1],time[2]]
 
    gha_a = cd.get_aries_gha(t)
    lha_a = gha_a + pos[1]

    s_z = nt.deg2rad(90.0 - s_h)
    s_psi = nt.deg2rad(90.0 - pos[0])

    # find declination
    cos_s_p = mat.cos(s_z)*mat.cos(s_psi) + mat.sin(s_z)*mat.cos(s_psi)*mat.cos(nt.deg2rad(s_w))
    if mat.fabs(cos_s_p) > 1.0: print('***WARNING: cos_p > 1.0 ({:.5f})'.format(cos_s_p))
    if cos_s_p > 1.0:
        cos_s_p = 1.0
    if cos_s_p < -1.0:
        cos_s_p = -1.0
    s_p = mat.acos(cos_s_p) 
    dec = 90.0 - nt.rad2deg(s_p)

    # find local hour angle
    cos_lha = (mat.cos(s_z) - mat.cos(s_p)*mat.cos(s_psi))/(mat.sin(s_p)*mat.sin(s_psi))
    if mat.fabs(cos_lha) > 1.0: print('***WARNING: cos_lha > 1.0 ({:.5f})'.format(cos_lha))
    if cos_lha > 1.0:
        cos_lha = 1.0
    if cos_lha < -1.0:
        cos_lha = -1.0
    lha = nt.rad2deg(mat.acos(cos_lha))
    if s_w < 180.0:
        lha = 360 - lha

    # find siderial hour angle
    sha = lha - lha_a
    if sha < 0:
        sha = 360 + sha
        
    return [dec,sha]

In [5]:
# poišči ime zvezde na podalgi izračunane deklinacije in sha
def findStarName(ns_db,pos,date,time,dec,sha):
    
    t = [date[2],date[1],date[0],time[0],time[1],time[2]]
    
    vs = findAllVisibleStars(t,pos,ns_db)
    
    dist = 1000.0
    ac = None
    for s in vs:
        dd = s['dec'] - dec
        ds = s['sha'] - sha
        da = mat.fabs(dd) + mat.fabs(ds)
        if da < dist:
            dist = da
            ac = s
            
    return ac

In [6]:
# load Astro database
cd = cdata.CelestialData(pp)
ns_db = cd.get_nav_stars_db()

In [7]:
# inputs
date = [21,2,2022]
time = [19,0,0]
fi = [46,6.08,'N']
la = [13,38.85,'E']
h = 20.0
pos = [nt.nav2dd(fi), nt.nav2dd(la), h]

# star measures
s_h = 51.0  # height
s_w = 182.0 # azimuth

print('MEASURED: alt={:s}, az={:s}'.format(nt.prettyPrintAlt(s_h), nt.prettyPrintAz(s_w)))
[dec, sha] = getStarDecAndSha(s_h, s_w, pos, date, time)
print('CALCULATED: dec={:s}, sha={:s}'.format(nt.prettyPrintDec(dec), nt.prettyPrintHA(sha)))
ss = findStarName(ns_db,pos,date,time,dec,sha) 
print('ALMANACH: name={:s}, dec={:s}, sha={:s}, alt={:s}, az={:s}'.
      format(ss['name'], nt.prettyPrintDec(ss['dec']), nt.prettyPrintHA(ss['sha']), nt.prettyPrintAlt(ss['alt']), nt.prettyPrintAz(ss['az'])))

MEASURED: alt=51°00.00′, az=182.00°
***WARNING: cos_lha > 1.0 (1.01563)
CALCULATED: dec=06°07.83′N, sha=269°40.65′
ALMANACH: name=Betelgeuse, dec=07°24.57′N, sha=270°54.49′, alt=51°17.62′, az=181.95°
